In [2]:
import pandas as pd
import numpy as np

df_num = pd.read_pickle("preprocessed")
df_num.head()

/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1085: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1085: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


,age,fnlwgt,capitalGain,capitalLoss,hoursPerWeek,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,relationship_Wife,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,sex_Female,sex_Male,salary_<=50K,salary_>50K
0,0.109589,0.144430,0.000000,0.0,0.397959,0,0,1,0,0,...,0,0,0,1,0,0,0,1,1,0
1,0.287671,0.051677,0.000000,0.0,0.500000,0,0,1,0,0,...,0,0,0,0,0,1,0,1,1,0
2,0.150685,0.219011,0.000000,0.0,0.397959,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,1
3,0.369863,0.099418,0.076881,0.0,0.397959,0,0,1,0,0,...,0,0,0,1,0,0,0,1,0,1
4,0.232877,0.125398,0.000000,0.0,0.295918,0,0,1,0,0,...,0,0,0,0,0,1,0,1,1,0


In [11]:
df_num.iloc[:, :-1].head()


,age,fnlwgt,capitalGain,capitalLoss,hoursPerWeek,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,relationship_Unmarried,relationship_Wife,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,sex_Female,sex_Male,salary_<=50K
0,0.109589,0.144430,0.000000,0.0,0.397959,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,1
1,0.287671,0.051677,0.000000,0.0,0.500000,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,1
2,0.150685,0.219011,0.000000,0.0,0.397959,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
3,0.369863,0.099418,0.076881,0.0,0.397959,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
4,0.232877,0.125398,0.000000,0.0,0.295918,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,1


In [18]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 5)
kmeans.fit_transform(df_num.iloc[:, :-2])

array([[3.04856398, 2.135173  , 2.87562645, 2.79393734, 2.47831919],
       [1.68487275, 2.13650092, 2.41828729, 1.27080577, 2.70711433],
       [1.73919746, 2.58936518, 2.79147844, 2.07441069, 3.04363382],
       ...,
       [2.983933  , 2.52408937, 1.72194744, 2.75373731, 2.17066049],
       [2.6549161 , 1.5967311 , 2.43297181, 2.37879951, 2.01055358],
       [2.49445411, 2.84225505, 2.12739017, 2.71445298, 2.62246172]])

In [20]:
kmeans.labels_[:50]

array([1, 3, 0, 3, 1, 0, 4, 3, 3, 0, 4, 3, 3, 1, 2, 2, 3, 3, 4, 1, 3, 0,
       1, 1, 3, 1, 0, 2, 1, 0, 2, 0, 1, 1, 1, 3, 0, 3, 3, 1, 2, 3, 3, 2,
       2, 1, 1, 4, 1, 1], dtype=int32)

+Spectral clustering, Gaussian mixtures, DBscan, isolation forest, EMM

In [ ]:
kmeans.